# Suggest closest news given a sample News article

In [9]:
# -*- coding: utf-8 -*-
import pandas as pd
import glob
import codecs
import numpy as np
import re
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import shuffle
from gensim.models import doc2vec

datas = pd.read_csv('goodNewsData')
datas.head()

,Unnamed: 0,labels,name,cleanText,noStopWords
0,1015,1,National News,﻿ २१ मंसिर चितवन टेण्डर बिनै बाल उद्यान स...,﻿ २१ मंसिर चितवन टेण्डर बिनै बाल उद्यान सञ्चाल...
1,2006,1,National News,﻿आश्विन १९ काठमाण्डौं बढा दशैं व्यस्तता क...,﻿आश्विन १९ काठमाण्डौं बढा दशैं व्यस्तता कमिआए ...
2,6224,1,National News,﻿संविधानसभा प्रमुख राजनीतिक दल निषेध गर्दै स...,﻿संविधानसभा प्रमुख राजनीतिक दल निषेध संविधानसभ...
3,11695,0,World,﻿एजेन्सी चीन र रूस मिलेर संसारकै सबैभन्दा...,﻿एजेन्सी चीन रूस मिलेर संसारकै सबैभन्दा शक्तिश...
4,10027,15,Economy,﻿ १ पुस वीरगन्ज भारत सरकार पाँच सय र हजार...,﻿ १ पुस वीरगन्ज भारत सरकार सय हजार नोट प्रतिबन...


In [4]:
allDirectories = ['World','National News','Auto','Entertainment','Sports','Education','Bank','Business Interview','Interview','Blog','Literature','Employment','Opinion','Technology','Tourism','Economy']

In [11]:
#load model we trained in the TrainWord2vec.ipynb
mymodel = doc2vec.Doc2Vec.load('w2vModel')

def cleanNews(rawNews,removeStopWords = False):
    #rawNews = rawNews.replace(','," ")
    #rawNews = re.sub(r'([a-z]+)(\n+)|\"*(\n+)|\'*(\n+) | \\*[a-z]*\s+'," ",rawNews,flags=re.IGNORECASE )
    #rawNews = re.sub(r'(\n)+', r' ' , rawNews,flags=re.IGNORECASE )
    rawNews = re.sub('\,|\"|\'| \)|\(|\)| \{| \}| \[| \]|!|‘|’|“|”| \:-|\?|।|/|\—', r' ', rawNews)
    rawNews = re.sub(r'(मा|को|ले|बाट|का|हरु|हरुसँग|सँग|लाई){1}\s+',r' ',rawNews)
    
    rawNews = re.sub(r'[a-zA-Z0-9]',r' ',rawNews)
    rawNews = rawNews.replace('\\','').replace('\n',' ').replace('\r',' ')
    if removeStopWords:
        rawNews = rawNews.split()
        rawNews =  [i for i in rawNews if i not in stopwordsList]
    return rawNews.split()
    


def showNews(rawNews,model,showtype = 'name',topn=10):
    
    '''
    showtype = ['name','text','model']
    rawNews takes news as string  ... ' १९ माघ, काठमाडौं । १३ औं ढोरपाटन महिला डबल लिग भलि '
    i.e
    
    news = ' १९ माघ, काठमाडौं । १३ औं ढोरपाटन महिला डबल लिग भलि '
    showNews(rawNews)
    
    returns the list of top most similar news to given news
    '''
    cleanerNews = cleanNews(rawNews)
    
    vectorValue = model.infer_vector(cleanerNews)

    similar = model.docvecs.most_similar([vectorValue],topn= topn)
    
    word = [i[0] for i in similar ]

    return datas[showtype][word].tolist()
    

#showNews(rawNews= raw,mymodel,'name',topn=6)

In [6]:
#testing on a newsfile i copid from onlinekhabar . (This is a sports news)
newsFile = '''२७ माघ, काठमाडौं । अन्तर्राष्ट्रिय क्रिकेट काउन्सिल (आईसीसी)ले विधान संशोधन गरेर चुनाव गर्न नेपाल क्रिकेट संघलाई ५० दिनको समय दिएको छ ।

हालै नेपाल आएको आईसीसीको टोलीले विधानमा देखिएको विवादबारे जानकारी लिएको थियो । लगत्तै राखेप, निर्वाचित क्यान र सल्लाहकार समूहका संयोजकले संयुक्त पत्रकार सम्मेलन गरेर क्रिकेट विवाद समाधानको मिलन बिन्दु पत्ता लगाएको घोषणा गरे ।

आईसीसीको बोर्ड बैठकले नेपाललाई विधान संशोधन गर्न मार्चसम्मको समय दिएको छ । अब निर्वाचित क्यानले तत्काल साधारणसभा बोलाएर सल्लाहकार समूहमा भएको सहमतिअनुसार क्रिकेट संघको विधान संशोधन गर्नु पर्नेछ ।

सल्लाहकार समूहले बनाएको विधानमा संशोधन गरेको भन्दै आईसीसीले अस्वीकार गरेको थियो । अब विधान संशोधन गर्दा चुनावी साधारणसभामा राखेपबाट ७ जना जाने र निर्वाचित क्यानका १७ जना स्वतः सहभागी हुने व्यवस्था हटाइनेछ ।

आईसीसीले विधान संशोधन भएर मार्च अन्तिमसम्ममा जिल्ला, प्रान्त र केन्द्रको चुनाव भइसक्ने अपेक्षा गरिएको बताएको छ । अहिले ललितपुर र भक्तपुर बाहेक सबै जिल्लाको चुनाव भइसकेको छ । तर, केही जिल्लामा विवाद भए अनुगमन समितिले हेरेर फेरि चुनाव गर्ने सहमति भएको छ ।

आईसीसीले भनेको छ- जुनमा हुने आईसीसीको साधारणसभामा नेपालको सदस्यताबारे निर्णय गर्नका लागि यो प्रक्रिया मार्च अन्त्यसम्ममा पूरा हुने अपेक्षा गरेका छौं ।'''

In [15]:
#Top 10 news similar to above sample sport news
showNews(newsFile,model=mymodel,showtype='name')
#we see , out of top 10 closest news , our trained model suggested sports news for most time

['Sports',
 'Sports',
 'National News',
 'Sports',
 'National News',
 'Sports',
 'National News',
 'Sports',
 'Sports',
 'Sports']

In [18]:
#lets view the news as an array of 10 news
showNews(newsFile,model=mymodel,showtype='cleanText')

#if you read the news below carefully, you can see it's related to cricket. 
#note that the whole training is completely unsupervised and we just gave the model text data .

['  \ufeffविश्व फूटबल सर्वोच्च निकाय फिफा सन् २०२६ विश्वकप टेन्डर प्रक्रिया स्थगित भए जनाए छ  फिफा विज्ञप्ति निकालेर टेन्डर प्रक्रिया स्थगित भए जानकारी गराए हो  केहिसता यता  फिफा भए भनिए व्यापक भ्रष्टाचार समाचार स्थान पाइरहे बेला यस्तो निर्णय बाहिर आए हो  सन् २०१० विश्वकप आयोजना गर्न दक्षिण अफ्रि दिन निर्णय गर्दा फिफा घुस लेनदेन भए प्रमाण मिलिरहे अन्तराष्ट्रिय सञ्चार माध्यम जनाइरहे छन्     यस्तै सन् २०१८ रसिया र सन् २०२२ कतार हुने विश्वकप आयोजक छनोट गर्दा फिफा पदाधिकारी घुस लिए आरोप लागिरहे छ  तर उन रुस सन् २०१८  हुन गइरहे विश्वकप भने पूर्व योजना अनुसार नै हुने बताए  ११ वर्ष पछि हुने विश्व कप फुटबल प्रतियोगिता कुन मुलुक आयोजना गर्ने भन्नेबारे घोषणा सन् २०१७ मे महिना क्वालालम्पुर गरिनेछ  तर  २०२६ विश्वकप आयोजक छनोट लागि टेन्डर नयाँ कार्यक्रम फिफा कार्यकारी समिति पछि गर्ने विज्ञप्ति उल्लेख छ   ',
 '  \ufeff १८ भदौ  काठमाडौं   अन्तराष्ट्रिय क्रिकेट काउन्सिल  आइसीसी  नेपाल क्रिकेट सघंमाथि लगाए आगामी नौ महिनाभित्र प्रतिवन्ध हट्ने बताए छ   यो अवधि क्रिकेट संघ विधान पुनरावलोन गरी नयाँ निर्वाच